In [1]:
import cadquery as cq
from cadquery import Vector as V
from cadquery import Location as L
from cadquery import Color as C
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show)
from jupyter_cadquery import set_sidecar, set_defaults
from cadquery import exporters
from cadquery import importers
import random

set_defaults(axes=False, grid=True, axes0=True, ortho=True, transparent=True)
set_sidecar("Gondolami", init=True)


Overwriting auto display for cadquery Workplane and Shape


TODO:

- point88
- top links
- wheel servo holder & gears
- point88 attachements
- front/back servo
- double caster
- wings
- solenoid
- marbles
- target pens
- weights


In [2]:
import math

def hexagon_h_to_r(h):
    return h * 2 / math.sqrt(3)

def hexagon_r_to_h(r):
    return r * math.sqrt(3) / 2

In [15]:
gondola_outer_diameter = 200
gondola_length = 200
arc_width = 25
rail_thickness = 5
thickness = 3


In [50]:

frontRingIn = (cq.Workplane('XY')
    .circle(gondola_outer_diameter/2)
    .circle((gondola_outer_diameter)/2-arc_width)
    .extrude(thickness))
frontRingIn.name = 'frontRingIn'

frontRingOut = (cq.Workplane('XY')
    .circle(gondola_outer_diameter/2-rail_thickness)
    .circle((gondola_outer_diameter)/2-arc_width+rail_thickness)
    .extrude(thickness))

link = cq.Workplane('XY').rect(arc_width, gondola_length).extrude(thickness)

m10_nut_width = 10
axe_diameter = 10
axe_length = gondola_length + 2 * m10_nut_width
axe = cq.Workplane('XZ').circle(axe_diameter/2).extrude(axe_length).translate(V(0, axe_length/2, 0))



In [25]:
from cadquery import importers
pens = importers.importStep('pens.step')

In [87]:
center_to_mid_arc = (gondola_outer_diameter-arc_width)/2
wheel_center = -center_to_mid_arc

front_wheel_offset = 50
back_wheel_offset = 50

assembly = (
    cq.Assembly(name='gondolami')
    .add(frontRingIn, loc=L(V(0, gondola_length/2, 0), V(1,0,0), 90), color=C("green"), name='back_ring_in')
    .add(frontRingIn, loc=L(V(0, -gondola_length/2, 0), V(1,0,0), 90), color=C("red"), name='front_ring_in')
    .add(frontRingOut, loc=L(V(0, gondola_length/2 + thickness, 0), V(1,0,0), 90), color=C("green"), name='back_ring_out')
    .add(frontRingOut, loc=L(V(0, -gondola_length/2 - thickness, 0), V(1,0,0), 90), color=C("red"), name='front_ring_out')
    .add(wheel, loc=L(V(0, gondola_length/2 - back_wheel_offset, wheel_center), V(1,0,0), 90), name='back_wheel')
    .add(wheel, loc=L(V(0, -gondola_length/2 + front_wheel_offset, wheel_center), V(1,0,0), 90), name='front_wheel')
    .add(link, loc=L(V(center_to_mid_arc, 0, 0)), color=C("yellow"), name='link')
    .add(link, loc=L(V(-center_to_mid_arc, 0, 0)), color=C("yellow"), name='link2')
    .add(axe, loc=L(V(0, 0, wheel_center)), color=C("blue"), name='axe')
    .add(point88, loc=L(V(0, point88_length/2, 0)), name='point88')
    .add(servo, loc=L(V(0, 0, 0), V(1, 0, 0), 90), name='servo')
    .add(wheel_gears, loc=L(V(0, 0, 0), V(1, 1, 0)), name='wheel_gears')
    #.add(pens, loc=L(V(0, 0, 0)), name='point88s')
)
assembly

AttributeError: 'Vector' object has no attribute 'zDir'

In [49]:
assembly.add(pens, loc=L(V(0, 0, 0)), name='point88s')
assembly

Done, using side car 'Gondolami'


![servo](http://sc01.alicdn.com/kf/HTB137beFVXXXXb7XXXXq6xXFXXXP/221432382/HTB137beFVXXXXb7XXXXq6xXFXXXP.jpg_.webp)

In [75]:
offsetX = -49.5 / 2 + 34.65
servo = (
    cq.Workplane('XY')
    .rect(40, 20).extrude(28.7)
    .faces(">Z").workplane().rect(54.5, 20).extrude(2.5)
    .faces(">Z").workplane().rect(49.5, 10, forConstruction=True).vertices().hole(5.0 )
    .faces(">Z").workplane().rect(40,20).extrude(37.0-2.5-28.7)
    .faces(">Z").workplane().moveTo(offsetX, 0).circle(15/2.0).extrude(1.5)
    .faces(">Z").workplane().moveTo(offsetX, 0).circle(10/2.0).extrude(1)
    .faces(">Z").workplane().moveTo(offsetX, 0).circle(5/2.0).extrude(7.1-1.5-1)
)
servo

Done, using side car 'Gondolami'


In [47]:
pens = cq.Assembly(name='pens')
#pens = Workplane('XZ')

posY = point88_length / 2
for i in range(n_pens):
    angle_deg = - i * 360.0 / float(n_pens) + 360.0 / 4.0
    angle = i * 2.0 * math.pi / float(n_pens)
    posX = center_to_mid_arc * math.cos(angle)
    posZ = wheel_center + center_to_mid_arc * math.sin(angle)
    pens.add(point88, loc=L(V(posX, posY, posZ), V(0, 1, 0), angle_deg), name=f'point88_{i}')
    #pens += point88.translate(V(posX, posY, posZ))

# exporters.export(pens, 'pens.step')

pens

Done, using side car 'Gondolami'


In [44]:
from cadquery import exporters


exporters.export(
            frontRing,
            'frontRing.svg',
            opt={
                "showAxes": False,
                "projectionDir": (0, 0, 1),
                "strokeWidth": 0.25,
                "strokeColor": (255, 0, 0),
                "hiddenColor": (0, 0, 255),
                "showHidden": True,
            },
        )

In [5]:
center_to_mid_arc = (gondola_outer_diameter-arc_width)/2
m3 = 3
wheel_thickness = 20
wheel_radius = center_to_mid_arc + wheel_thickness / 2
wheel_diameter = 2 * wheel_radius
n_pens = 20
n_spokes = 4
spoke_width = 10
inner_diameter = wheel_diameter - 2 * wheel_thickness

point88_diameter = 8;
point88_h = hexagon_r_to_h(point88_diameter / 2)

spokes = cq.Workplane('XY').rect(inner_diameter, spoke_width).extrude(thickness)

for i in range(int(n_spokes/2)):
    spokes += cq.Workplane('XY').rect(inner_diameter, spoke_width).extrude(thickness).rotateAboutCenter(V(0,0,1), i*360/n_spokes)

wheel = (cq.Workplane('XY')
    .circle(wheel_diameter/2)
    .circle(inner_diameter/2)
    .polarArray(wheel_diameter/2-wheel_thickness/2, 0, 360, n_pens)
    .polygon(6, point88_diameter)
    .extrude(thickness))

wheel += spokes

wheel = wheel.faces('>Z').hole(m3)

wheel

Done, using side car 'Gondolami'


In [6]:
from cadquery import Workplane, Edge, Wire, Vector
from math import *

#    r_ref = m*z/2
#    r_ref = (p/pi)*z/2
#    z = 2 * r_ref * pi / p
#    z = 2 * r_ref * pi / p
#    z = number of teeth

# m = p / pi
# p = m * pi

def involute_gear(m, z, alpha=20, shift=0, N=20):
    '''
    See https://khkgears.net/new/gear_knowledge/gear_technical_reference/involute_gear_profile.html
    for math
    '''
    
    alpha = radians(alpha)

    # radii
    r_ref = m*z/2
    r_top = r_ref + m*(1+shift)
    r_base = r_ref*cos(alpha)
    r_d = r_ref - 1.25*m
    
    inv = lambda a: tan(a) - a
    
    # angles of interest
    alpha_inv = inv(alpha)
    alpha_tip = acos(r_base/r_top)
    alpha_tip_inv = inv(alpha_tip)
    
    a = 90/z+degrees(alpha_inv)
    a2 = 90/z++degrees(alpha_inv)-degrees(alpha_tip_inv)
    a3 = 360/z-a
    
    # involute curve (radius based parametrization)
    def involute_curve(r_b,sign=1):
        
        def f(r):
            alpha = sign*acos(r_b/r)
            x = r*cos(tan(alpha) - alpha) 
            y = r*sin(tan(alpha) - alpha)
        
            return x,y
        
        return f
    
    # construct all the profiles
    right = (
        Workplane()
        .transformed(rotate=(0,0,a))
        .parametricCurve(involute_curve(r_base,-1), start=r_base, stop = r_top, makeWire=False, N=N)
        .val()
    )
    
    left = (
        Workplane()
        .transformed(rotate=(0,0,-a))
        .parametricCurve(involute_curve(r_base), start=r_base, stop = r_top, makeWire=False, N=N)
        .val()
    )

    top = Edge.makeCircle(r_top, angle1=-a2, angle2=a2)
    bottom = Edge.makeCircle(r_d, angle1=-a3, angle2=-a)
    
    side = Edge.makeLine( cq.Vector(r_d,0), cq.Vector(r_base,0))
    side1 = side.rotate(cq.Vector(0, 0, 0), cq.Vector(0, 0, 1), -a)
    side2 = side.rotate(cq.Vector(0, 0, 0), cq.Vector(0, 0, 1), -a3)
    
    # single tooth profile
    profile = Wire.assembleEdges([left,top,right,side1,bottom,side2])
    profile = profile.chamfer2D(m/4, profile.Vertices()[-3:-1])

    # complete gear
    res = (
        Workplane()
        .polarArray(0,0,360,z)
        .each(lambda loc: profile.located(loc))
        .consolidateWires()
    )

    return res.val()

# with gear_module = 1 (= pitch / math.pi) the gear diameter is equal 
# to the number of teeth!
gear_module = 1
gear_pitch = gear_module * math.pi
gear_n_teeth = 20
gear_ref_radius = gear_n_teeth * gear_module / 2
print(gear_ref_radius)
gear = Workplane(obj=involute_gear(gear_module, gear_n_teeth)).toPending().extrude(thickness)
gear

10.0
Done, using side car 'Gondolami'


In [76]:
gear_module = 1
gear_pitch = gear_module * math.pi
gear_wanted_diameter = 50
gear_n_teeth = int((gear_wanted_diameter / 2) // (gear_module / 2))
print(gear_n_teeth)
gear_ref_radius = gear_n_teeth * gear_module / 2
print(gear_ref_radius)
wheel_gear = Workplane(obj=involute_gear(gear_module, gear_n_teeth)).toPending().extrude(thickness).faces(">Z").hole(10)
wheel_gear

50
25.0
Done, using side car 'Gondolami'


In [77]:
gear_wanted_diameter = 25
gear_n_teeth = int((gear_wanted_diameter / 2) // (gear_module / 2))
print(gear_n_teeth)
gear_ref_radius = gear_n_teeth * gear_module / 2
print(gear_ref_radius)
wheel_servo_gear = Workplane(obj=involute_gear(gear_module, gear_n_teeth)).toPending().extrude(thickness).faces(">Z").hole(3)
wheel_servo_gear

25
12.5
Done, using side car 'Gondolami'


In [81]:
wheel_gears = cq.Assembly(name='wheel_gears')
wheel_gears.add(wheel_gear, loc=L(V(0, 0, 0)), color=C("green"), name='wheel_gear')
wheel_gears.add(wheel_servo_gear, loc=L(V(50/2+25/2, 0, 0)), color=C("blue"), name='wheel_servo_gear')
wheel_gears

Done, using side car 'Gondolami'


In [7]:
# Point88

point88_length = 166
point88_cap_length = 23
point88_diameter = 8
point88_h = hexagon_r_to_h(point88_diameter / 2)

point88 = (
    Workplane('XZ')
    .polygon(6, point88_diameter).extrude(point88_length - point88_cap_length)
    .faces('<Y')
    .circle(point88_diameter/2-0.5).extrude(point88_cap_length - 5, taper=4)
    .faces('<Y')
    .circle(1).extrude(5, taper=4)
)
point88

Done, using side car 'Gondolami'
